In [2]:
%config Completer.use_jedi = False

/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  if __name__ == '__main__':


In [3]:
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('models.py') ) ))
import pandas as pd
from transformers import BertTokenizer
import six
import re
import json
import numpy as np
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from seqeval.metrics import classification_report as classification_report_seqeval

import tensorflow as tf
from tensorflow import keras
from transformers import TFElectraModel, TFBertModel, AutoTokenizer
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Activation, Lambda, Input
from tf2crf import CRF, ModelWithCRFLoss

import argparse
import time
from model.models import TFElectraClassifier
import nltk
import openpyxl
import gc

import random
from tensorflow.keras.utils import to_categorical

/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.4.0 and strictly below 2.7.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [4]:
tokenizer = BertTokenizer.from_pretrained('../vocab-NER.txt', do_lower_case=False)
kos_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1646: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [5]:
label_file = pd.read_csv('../model/labels.txt', header=None, names=['code'], index_col=False)
label_file

,code
0,EA01
1,EA02
2,EA03
3,EA04
4,EA05
...,...
140,ND05
141,ND06
142,ND07
143,ND08


In [6]:
idx2lab = {key: label_file['code'][key] for key in range(len(label_file))}
idx2lab
#label 144 개

{0: 'EA01',
 1: 'EA02',
 2: 'EA03',
 3: 'EA04',
 4: 'EA05',
 5: 'EA06',
 6: 'EA07',
 7: 'EA08',
 8: 'EA09',
 9: 'EA10',
 10: 'EA11',
 11: 'EA13',
 12: 'EA14',
 13: 'EA15',
 14: 'EB01',
 15: 'EB02',
 16: 'EB03',
 17: 'EB04',
 18: 'EB05',
 19: 'EB06',
 20: 'EB07',
 21: 'EC01',
 22: 'EC02',
 23: 'EC03',
 24: 'EC04',
 25: 'EC05',
 26: 'EC06',
 27: 'EC07',
 28: 'EC09',
 29: 'ED01',
 30: 'ED02',
 31: 'ED03',
 32: 'ED04',
 33: 'ED05',
 34: 'ED06',
 35: 'ED07',
 36: 'ED08',
 37: 'ED09',
 38: 'ED10',
 39: 'ED11',
 40: 'EE01',
 41: 'EE02',
 42: 'EE03',
 43: 'EE04',
 44: 'EE05',
 45: 'EE06',
 46: 'EE07',
 47: 'EE08',
 48: 'EE09',
 49: 'EE10',
 50: 'EE11',
 51: 'EE12',
 52: 'EE13',
 53: 'EE14',
 54: 'EF01',
 55: 'EF02',
 56: 'EF04',
 57: 'EF05',
 58: 'EF06',
 59: 'EG04',
 60: 'EG06',
 61: 'EG07',
 62: 'EH01',
 63: 'EH02',
 64: 'EH03',
 65: 'EH06',
 66: 'EH07',
 67: 'EH09',
 68: 'EH10',
 69: 'EH11',
 70: 'EI01',
 71: 'EI02',
 72: 'EI03',
 73: 'EI04',
 74: 'EI05',
 75: 'EI06',
 76: 'EI08',
 77: 'EI0

In [8]:
lab2idx = {value:key for key,value in idx2lab.items()}
lab2idx

{'EA01': 0,
 'EA02': 1,
 'EA03': 2,
 'EA04': 3,
 'EA05': 4,
 'EA06': 5,
 'EA07': 6,
 'EA08': 7,
 'EA09': 8,
 'EA10': 9,
 'EA11': 10,
 'EA13': 11,
 'EA14': 12,
 'EA15': 13,
 'EB01': 14,
 'EB02': 15,
 'EB03': 16,
 'EB04': 17,
 'EB05': 18,
 'EB06': 19,
 'EB07': 20,
 'EC01': 21,
 'EC02': 22,
 'EC03': 23,
 'EC04': 24,
 'EC05': 25,
 'EC06': 26,
 'EC07': 27,
 'EC09': 28,
 'ED01': 29,
 'ED02': 30,
 'ED03': 31,
 'ED04': 32,
 'ED05': 33,
 'ED06': 34,
 'ED07': 35,
 'ED08': 36,
 'ED09': 37,
 'ED10': 38,
 'ED11': 39,
 'EE01': 40,
 'EE02': 41,
 'EE03': 42,
 'EE04': 43,
 'EE05': 44,
 'EE06': 45,
 'EE07': 46,
 'EE08': 47,
 'EE09': 48,
 'EE10': 49,
 'EE11': 50,
 'EE12': 51,
 'EE13': 52,
 'EE14': 53,
 'EF01': 54,
 'EF02': 55,
 'EF04': 56,
 'EF05': 57,
 'EF06': 58,
 'EG04': 59,
 'EG06': 60,
 'EG07': 61,
 'EH01': 62,
 'EH02': 63,
 'EH03': 64,
 'EH06': 65,
 'EH07': 66,
 'EH09': 67,
 'EH10': 68,
 'EH11': 69,
 'EI01': 70,
 'EI02': 71,
 'EI03': 72,
 'EI04': 73,
 'EI05': 74,
 'EI06': 75,
 'EI08': 76,
 'EI09': 

In [9]:
df = pd.read_csv("/home/ubuntu/JIHO/NTIS/data/keywords_code_data.csv",index_col=False)
len(df)

1298573

In [12]:
df.head()

,Unnamed: 0,keyword,code
0,0,Hydrogen,EC01
1,1,Purification,EC01
2,2,Fuel cell,EC01
3,3,Silica,EC01
4,4,Separation,EC01


In [41]:
df_drop = df.dropna(axis=0)

In [40]:
len(df)

1298573

In [37]:
for k in tqdm(df_drop.iloc):
    if type(k['keyword']) != str:
        print(k['keyword'])

1522it [00:00, 7620.48it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


3059it [00:00, 7656.81it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


4595it [00:00, 7653.88it/s]

...

132527it [00:17, 7654.03it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


134491it [00:17, 7710.93it/s]

nan
nan


KeyboardInterrupt: 

In [42]:
multi_keywords = []
multi_codes = []
for k in tqdm(df_drop.iloc):
    if len(k['keyword'].split()) >= 2:
        multi_keywords.append(k['keyword'])
        multi_codes.append(k['code'])

1294248it [02:54, 7428.91it/s]


In [43]:
print(len(multi_keywords),len(multi_codes))
#멀티 키워드 , 라벨 수

568302 568302


In [11]:
keywords = []
codes = []
for k in tqdm(df.iloc):
    keywords.append(k['keyword'])
    codes.append(k['code'])
len(codes)

1298573it [02:51, 7553.32it/s]


1298573

토크나이저 확인

In [52]:
print(kos_tokenizer.tokenize(multi_keywords[4]))
print(tokenizer.tokenize(multi_keywords[4]))

['histone', 'acetylation']
['Histone', 'acetylation']


In [53]:
sci_tokenized_multi = []
tokenized_multi = []
for k in multi_keywords[:100]:
    sci_tokenized_multi.append(kos_tokenizer.tokenize(k))
    tokenized_multi.append(tokenizer.tokenize(k))
print(sci_tokenized_multi)
print(tokenized_multi)

[['fuel', 'cell'], ['taste', 'sensation'], ['in', 'vivo', 'imaging'], ['multi', '##photon', 'microscopy'], ['histone', 'acetylation'], ['chromatin', 'loop', '##ing'], ['human', 'beta', 'glob', '##in', 'locus'], ['loop', 'mediating', 'protein'], ['hair', 'cell'], ['neural', 'circuit'], ['fusion', 'technology'], ['implant', '##able', 'ic'], ['bio', '-', 'information', 'processing'], ['stem', 'cells'], ['artificial', 'transcription', 'factor'], ['auditory', 'organ'], ['inner', 'ear'], ['middle', 'ear'], ['neural', 'crest'], ['mouse', 'model'], ['integrin', 'alpha', 'm'], ['innate', 'immune'], ['plant', 'disease'], ['fungal', 'pathogen'], ['rice', 'blast', 'disease'], ['plant', 'disease'], ['fungal', 'pathogen'], ['rice', 'blast', 'disease'], ['hepatitis', 'b', 'virus'], ['chronic', 'hepatitis', 'b'], ['drug', 'resistance'], ['antiviral', 'effect'], ['3', '-', 'dimensional', 'numerical', 'analysis'], ['local', 'sc', '##our'], ['river', 'management'], ['had', '##ley', 'circulation'], ['atmo

label idx 로 변환

In [56]:
code2idx = []
for k in multi_codes:
    code2idx.append(lab2idx[k])

568302

In [57]:
print(len(multi_keywords),len(code2idx))

568302 568302


Tokenizer

In [58]:
MAX_LENGTH = 256

In [61]:
encode_train_data = []
input_ids = []
attention_masks = []
token_type_ids =[]
for line in tqdm(multi_keywords):
    encoded_dict = tokenizer.encode_plus(line,\
                                        add_special_tokens=True,\
                                        pad_to_max_length=True,\
                                        max_length = MAX_LENGTH,\
                                        return_attention_mask=True,\
                                        truncation =True)
    
    input_id = encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

train_input_ids=np.array(input_ids, dtype=int)
train_attention_masks=np.array(attention_masks, dtype=int)
train_token_type_ids=np.array(token_type_ids, dtype=int)
train_x=(train_input_ids, train_attention_masks, train_token_type_ids)

  0%|          | 0/568302 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 568302/568302 [02:04<00:00, 4561.66it/s]


In [ ]:
keywords 